In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import zipfile

# Path to your zip file (replace with your file's path)
zip_file_path = '/content/drive/MyDrive/OSCD.zip'
dataset_folder = '/content/drive/MyDrive/OSCD/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_folder)

print(f"Unzipped to: {dataset_folder}")

Unzipped to: /content/drive/MyDrive/OSCD/


In [21]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

train_folder = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/images/train2017')
val_folder = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/images/val2017')
train_annotation = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/annotations/instances_train2017.json')
val_annotation = os.path.join(dataset_folder, 'coco_carton/oneclass_carton/annotations/instances_val2017.json')


transform = transforms.Compose([
    # transforms.Resize((512, 512)),
    transforms.ToTensor()
])

train_dataset = datasets.CocoDetection(root=train_folder, annFile=train_annotation, transform=transform)
val_dataset = datasets.CocoDetection(root=val_folder, annFile=val_annotation, transform=transform)

loading annotations into memory...
Done (t=2.19s)
creating index...
index created!
loading annotations into memory...
Done (t=0.18s)
creating index...
index created!


In [22]:
def collate_fn(batch):
    images, targets = list(zip(*batch))
    return images, targets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=collate_fn)

In [23]:
train_data = next(iter(train_loader))
val_data = next(iter(val_loader))

In [32]:
img = train_data[0][12]
target = train_data[1][12]
print(img.shape)
print(target)

torch.Size([3, 500, 500])
[{'id': 67327, 'image_id': 3372, 'category_id': 1, 'segmentation': [[180.2474226804124, 150.51546391752578, 0.0, 145.0, 0.0, 5.0, 176.63917525773195, 13.402061855670103]], 'bbox': [0, 5, 180.2474226804124, 145.51546391752578], 'iscrowd': 0, 'area': 26228.787331278567}, {'id': 67328, 'image_id': 3372, 'category_id': 1, 'segmentation': [[179.21649484536084, 13.917525773195877, 185.40206185567013, 152.57731958762886, 499.0, 153.0, 499.0, 17.0]], 'bbox': [179.21649484536084, 13.917525773195877, 319.78350515463916, 139.08247422680412], 'iscrowd': 0, 'area': 44476.281113827186}, {'id': 67329, 'image_id': 3372, 'category_id': 1, 'segmentation': [[183.340206185567, 151.54639175257734, 185.9175257731959, 275.77319587628864, 499.0, 277.0, 499.0, 155.0]], 'bbox': [183.340206185567, 151.54639175257734, 315.659793814433, 125.45360824742266], 'iscrowd': 0, 'area': 39600.660112658086}, {'id': 67330, 'image_id': 3372, 'category_id': 1, 'segmentation': [[180.76288659793818, 15

In [ ]:
!pip install pycocotools

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from utils import polygons_to_mask

def show_image_with_masks(image, target, image_height, image_width):
    img = img.permute(1, 2, 0).cpu().numpy()

    plt.figure(figsize=(12,12))
    plt.imshow(img)

    for obj in target:
        segmentation = obj['segmentation']
        mask = polygons_to_mask(segmentation, image_height, image_width)
        plt.imshow(mask, alpha=0.5, cmap='jet')

    plt.axis('off')
    plt.show()